<div style="display: flex; align-items: center; justify-content: center; flex-wrap: wrap;">
    <div style="flex: 1; max-width: 400px; display: flex; justify-content: center;">
        <img src="https://i.ibb.co/JBPWVYR/Logo-Nova-IMS-Black.png" style="max-width: 50%; height: auto; margin-top: 50px; margin-bottom: 50px;margin-left: 6rem;">
    </div>
    <div style="flex: 2; text-align: center; margin-top: 20px;margin-left: 8rem;">
        <div style="font-size: 28px; font-weight: bold; line-height: 1.2;">
            <span style="color: #22c1c3;">DL Project |</span> <span style="color: #08529C;">Predicting Rare Species from Images using Deep Learning</span>
        </div>
        <div style="font-size: 17px; font-weight: bold; margin-top: 10px;">
            Spring Semester | 2024 - 2025
        </div>
        <div style="font-size: 17px; font-weight: bold;">
            Master in Data Science and Advanced Analytics
        </div>
        <div style="margin-top: 20px;">
            <div>André Silvestre, 20240502</div>
            <div>Diogo Duarte, 20240525</div>
            <div>Filipa Pereira, 20240509</div>
            <div>Maria Cruz, 20230760</div>
            <div>Umeima Mahomed, 20240543</div>
        </div>
        <div style="margin-top: 20px; font-weight: bold;">
            Group 37
        </div>
    </div>
</div>

<div style="background: linear-gradient(to right, #22c1c3, #27b1dd, #2d9cfd, #090979); 
            padding: 1px; color: white; border-radius: 500px; text-align: center;">
</div>

## **📚 Libraries Import**

In [ ]:
!pip install visualkeras

In [ ]:
# System imports
import os
import sys
import time
import datetime
from tqdm import tqdm
from typing_extensions import Self, Any      # For Python 3.10
# from typing import Self, Any               # For Python >3.11

from pathlib import Path

# Data manipulation imports
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Data visualization imports
import matplotlib.pyplot as plt
import seaborn as sns

# Deep learning imports
import tensorflow as tf
from keras.ops import add
from keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Model, Sequential, Input
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling, Lambda, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras import regularizers                                                                           # For L2 regularization
import visualkeras

# Evaluation imports
from keras.metrics import CategoricalAccuracy, AUC, F1Score, Precision, Recall

# Other imports
from itertools import product

# Set the style of the visualization
pd.set_option('future.no_silent_downcasting', True)   # use int instead of float in DataFrame
pd.set_option("display.max_columns", None)            # display all columns

# Disable warnings (FutureWarning)
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# Set random seed for reproducibility
np.random.seed(2025)

In [ ]:
#?

In [ ]:
#?

In [ ]:
#?

In [ ]:
# Import custom module for importing data, visualization, and utilities
import utilities

<div style="background: linear-gradient(to right, #22c1c3, #27b1dd, #2d9cfd, #090979);
            padding: 1px; color: white; border-radius: 500px; text-align: center;">
</div>

## **🧮 Import Databases**

In [ ]:
# Run in Google Collab to download the dataset already splitted
# Source: https://stackoverflow.com/questions/25010369/wget-curl-large-file-from-google-drivez
# Download the file from Google Drive using wget
!wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate \
  "https://drive.usercontent.google.com/download?id=11vkRJLP-re8E-8DWaoKeSuG66u64ez0J&export=download" -O- | \
  sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p' > /tmp/confirm.txt

# Read the confirmation token from the temporary file
with open('/tmp/confirm.txt', 'r') as f:
    confirm_token = f.read().strip()

# Download the file using the confirmation token and cookies
!wget --load-cookies /tmp/cookies.txt \
  "https://drive.usercontent.google.com/download?id=11vkRJLP-re8E-8DWaoKeSuG66u64ez0J&export=download&confirm={confirm_token}" \
  -O /content/RareSpecies_Split.zip

# Clean up temporary files
!rm /tmp/cookies.txt /tmp/confirm.txt

# List files in the /content directory to verify the download
!ls -lh /content/

# Unzip the downloaded file
!unzip /content/RareSpecies_Split.zip -d /content/

# List the unzipped files to verify
!ls -lh /content/

In [ ]:
# Define the path to the data (for Google Collab)
train_dir = Path("/content/RareSpecies_Split/train")
val_dir = Path("/content/RareSpecies_Split/val")
test_dir = Path("/content/RareSpecies_Split/test")

<div style="background: linear-gradient(to right,rgb(255, 252, 58),rgb(253, 173, 45),rgb(190, 64, 5),rgb(190, 64, 5));
            padding: 1px; color: black; border-radius: 500px; text-align: left;">

image_size = (384, 384)   # EfficientNetV2S input size <br>
batch_size = 24           # Reduced batch size - EfficientNetV2S is memory intensive <br>
</div>

In [ ]:
# Image Generators
n_classes = 202                                     # Number of classes (we already know this based on previous notebook)
image_size = (384, 384)                             # Image size (384x384)
img_height, img_width = image_size                  # Image dimensions
batch_size = 24                                     # Batch size
input_shape = (img_height, img_width, 3)            # Input shape of the model
value_range = (0.0, 1.0)                            # Range of pixel values

In [ ]:
# Get class names from directory
class_names = sorted(os.listdir(train_dir))
class_indices = {name: i for i, name in enumerate(class_names)}

# Import the image dataset from the directory
from utilities import load_images_from_directory
train_datagen, val_datagen, test_datagen = load_images_from_directory(train_dir, val_dir, test_dir,
                                                                      labels='inferred', label_mode='categorical',
                                                                      class_names=class_names, color_mode='rgb',
                                                                      batch_size=batch_size, image_size=image_size, seed=2025,
                                                                      interpolation='bilinear', crop_to_aspect_ratio=False, pad_to_aspect_ratio=False)

print(f"\nLoaded: Train ({train_datagen.cardinality().numpy() * batch_size}), "
        f"Val ({val_datagen.cardinality().numpy() * batch_size}), "
        f"Test ({test_datagen.cardinality().numpy() * batch_size})")

In [ ]:
# Check the shape of the data (batch_size, img_width, img_height, 3)
for x, y in train_datagen.take(1):
    print("Train batch shape:", x.shape, y.shape)
for x, y in val_datagen.take(1):
    print("Val batch shape:", x.shape, y.shape)
for x, y in test_datagen.take(1):
    print("Test batch shape:", x.shape, y.shape)

# <a class='anchor' id='3'></a>
<br>
<style>
@import url('https://fonts.cdnfonts.com/css/avenir-next-lt-pro?styles=29974');
</style>

<div style="background: linear-gradient(to right, #22c1c3, #27b1dd, #2d9cfd, #090979); 
            padding: 10px; color: white; border-radius: 300px; text-align: center;">
    <center><h1 style="margin-left: 140px;margin-top: 10px; margin-bottom: 4px; color: white;
                       font-size: 32px; font-family: 'Avenir Next LT Pro', sans-serif;">
        <b>3 | Modeling - EfficientNetV2S</b></h1></center>
</div>

<br><br>

<center><img src="" style="width: 700px"></center>

`EfficientNetV2S` is a member of the `EfficientNetV2` family, an enhanced version of the original `EfficientNet` models. It's known for balancing speed and accuracy. The "S" indicates its size (small).
Key improvements include:

`Progressive Learning`: Gradually increasing image size and regularization during training; <br>
`Efficient Architecture`: Using a combination of MBConv and faster Fused-MBConv blocks.

<div style="background: linear-gradient(to right,rgb(255, 252, 58),rgb(253, 173, 45),rgb(190, 64, 5),rgb(190, 64, 5));
            padding: 0px; color: black; border-radius: 5px; text-align: left;">

4 parâmetros: <br>
weights='imagenet'/=None/='path' <br>
include_top=False <br>
input_shape=input_shape<br>
pooling='avg'/='max'/None <br><br>

Ir testando o dropout
</div>


# **💡 Modeling**

In [ ]:
# from tensorflow.keras.applications import EfficientNetV2S

# # Load pre-trained EfficientNetV2S
# EfficientNet_model = EfficientNetV2S(
#     weights='imagenet',  # Initializes the model with weights pre-trained on ImageNet (crucial for transfer learning)
#     ### strong starting point, allows to learn faster and often perform better, especially with limited data
#     ### Alternatives:
#     ### weights=None: Starts with random initialization. Useful if you're not doing transfer learning and want to train from scratch
#     ### weights='path/to/weights.h5': Loads weights from a specific file. Useful if you have custom pre-trained weights or want to resume training from a checkpoint.
#     include_top=False,  # Removes the classification layer (designed for 1000 classes)
#     input_shape=input_shape, #Input Shape adjusted for EfficientNetV2S (RGB images of size 384x384)
#     pooling='avg'  # Use average pooling after the convolutional layers to reduce dimensionality before passing to the fully connected layers
#     ### prevents overfitting by reducing the number of parameters in the model
#     ### creates a more robust representation of the input data
#     ### Alternatives:
#     ### pooling='max': Uses max pooling instead of average pooling. This can be useful if you want to retain the most prominent features from the feature maps.
#     ### pooling=None: No pooling is applied. This means the output will be the raw feature maps from the last convolutional layer. This increase the computational cost
# )



# # Freeze the weights of the pre-trained EfficientNet model
# ### TRANSFER LEARNING(especially when you have limited data):
# ### weights in these layers will not be updated during training
# ### It allows you to leverage the knowledge learned from ImageNet without risking overfitting to your smaller dataset
# ### ---------------------------------------------------------------------------
# ###  You might choose to unfreeze some or all of these layers later (FINE-TUNING) for better performance once the custom head is trained
# EfficientNet_model.trainable = False #Freezing base model. We might want to unfreeze it later on after training the top layers


# # Create a new model on top
# inputs = keras.Input(shape=input_shape) #define the input layer (tuple:H,W,C) of your new model


# ### passing the input tensor through the pre-trained EfficientNet
# ### -------------------------------------------------------------------
# ### training=False ensures that the Batch Normalization layers within EfficientNet behave in inference mode.??????????? Batch Normalization calculates statistics differently during training and inference
# ### If training=True (even with the model frozen), the Batch Normalization layers would recalculate and update their internal statistics based on your smaller training dataset, potentially hurting performance
# x = EfficientNet_model(inputs, training=False) #Important to set training to False to avoid updating weights when extracting features


# ### This adds a dropout layer after the EfficientNet
# ### Dropout is a regularization technique that randomly sets a fraction of the input units to 0 during training. This helps prevent overfitting by reducing the model's reliance on any single neuron
# x = keras.layers.Dropout(0.2)(x) #Regularization (Dropout)


# # We removed the first dense layer since average pooling creates a good enough output to only require one more fully-connected layer
# ### Create the classification head of your model --> dense (fully connected) layer with n_classes output units (one for each species you want to classify)
# ### ----------------------------------------------------------------------------------------------------------------------------------
# ### Softmax activation function ensures that the outputs are probabilities that sum to 1
# outputs = keras.layers.Dense(n_classes, activation="softmax")(x)



# # Creates the Keras model, specifying the inputs and outputs tensors
# model = keras.Model(inputs, outputs)


# model.summary()

In [ ]:
from tensorflow.keras.applications import EfficientNetV2S


class RareSpeciesEfficientNet(Model):
    def __init__(self, n_classes=202, freeze_base=True):  # Add freeze_base argument
        
        super().__init__()
        self.n_classes = n_classes
        self.freeze_base = freeze_base

        self.rescale_layer = Rescaling(scale=1.0 / 255.0, name="Rescale_Layer")

        self.base_model = EfficientNetV2S(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape,
            pooling='avg'  # Important for feature extraction
        )

        self.base_model.trainable = not freeze_base #Freeze or unfreeze based on constructor argument


        self.dropout1 = Dropout(0.2)
        self.dense1 = Dense(128, activation='relu')
        self.bn_dense1 = BatchNormalization()
        self.dropout2 = Dropout(0.5)
        self.dense_output = Dense(n_classes, activation='softmax')



    def call(self, inputs, training=False):
        x = self.rescale_layer(inputs)
        x = self.base_model(x, training=False) 
        x = self.dropout1(x, training=training)  # Regularization after EfficientNet

        x = self.dense1(x)
        x = self.bn_dense1(x, training=training)
        x = self.dropout2(x, training=training)
        outputs = self.dense_output(x)
        return outputs

In [ ]:
#Training
model = RareSpeciesEfficientNet(n_classes=n_classes, freeze_base=True)  # Start with frozen base
inputs = Input(shape=input_shape)    
_ = model.call(inputs) 
model.summary()

In [ ]:
#-----FEATURE EXTRACTION-----#
optimizer = Adam(learning_rate=0.001, weight_decay=0.01, name="Optimizer")

loss = CategoricalCrossentropy(name="Loss")

metrics = [CategoricalAccuracy(name="accuracy"),
           Precision(name="precision"), 
           Recall(name="recall"),
           F1Score(average="macro", name="f1_score"),
           AUC(name="auc")]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# Callbacks

model_name = f"RareSpeciesEfficientNet_FeatureExtraction_{datetime.datetime.now().strftime('%Y%m%d')}" 
callbacks = [
    ModelCheckpoint(f"./ModelCallbacks/checkpoint_{model_name}.keras", monitor="val_loss", save_best_only=True, verbose=0),
    CSVLogger(f"./ModelCallbacks/metrics_{model_name}.csv"),
    LearningRateScheduler(lambda epoch, lr: lr * 0.95),
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
]

In [ ]:
# Train model
start_time = time.time()
history = model.fit(train_datagen, batch_size = batch_size, epochs=10, validation_data=val_datagen, callbacks=callbacks, verbose=1)
train_time = round(time.time() - start_time, 2)

print(f"\nTraining completed in \033[1m{train_time} seconds ({str(datetime.timedelta(seconds=train_time))} h)\033[0m).")

In [ ]:
#-----FINE-TUNING-----#

model.base_model.trainable = True # Unfreeze the EfficientNet base
model.summary(show_trainable=True)  # Show that layers are now trainable

# Compile model
optimizer_fine_tune = tf.keras.optimizers.Adam(1e-5)  # Lower learning rate for fine-tuning
model.compile(optimizer=optimizer_fine_tune, loss=loss, metrics=metrics) # Recompile with new optimizer



model_name = f"RareSpeciesEfficientNet_FineTuning_{datetime.datetime.now().strftime('%Y%m%d')}"  # Update model name
callbacks_finetune = [ # New callbacks for fine-tuning (potentially different patience, etc.)
    ModelCheckpoint(f"./ModelCallbacks/checkpoint_{model_name}.keras", monitor="val_loss", save_best_only=True, verbose=0),
    CSVLogger(f"./ModelCallbacks/metrics_{model_name}.csv"),
    EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True) # Possibly lower patience for fine-tuning
]


history_fine_tuning = model.fit(
    train_datagen, batch_size=batch_size, epochs=5,
    validation_data=val_datagen, callbacks=callbacks_finetune, verbose=1
) # Fine-tune the whole model






In [ ]:
# Create a directory for saving the model and logs
model_name = f"RareSpeciesEfficientNet_{datetime.datetime.now().strftime('%Y%m%d')}"                                                                             # Model name
print(f"\n\033[1mModel name:\033[0m {model_name}")

In [ ]:

# Load pre-trained EfficientNetV2S
EfficientNet_model = EfficientNetV2S(
    weights='imagenet',  # Initializes the model with weights pre-trained on ImageNet (crucial for transfer learning)
    include_top=False,  # Do not include the ImageNet classifier at the top (it was made for 1000 classes)
    input_shape=input_shape, #Input Shape adjusted for EfficientNetV2S
    pooling='avg'  # Use average pooling to reduce dimensionality
)

# Freeze the base model's layers
EfficientNet_model.trainable = False #Freezing base model. We might want to unfreeze it later on after training the top layers

# Create a new model on top
inputs = keras.Input(shape=input_shape)

x = EfficientNet_model(inputs, training=False) #Important to set training to False to avoid updating weights when extracting features

x = keras.layers.Dropout(0.2)(x) #Regularization (Dropout)
# We removed the first dense layer since average pooling creates a good enough output to only require one more fully-connected layer
outputs = keras.layers.Dense(n_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs)



model.summary()

In [ ]:
# Visualize the model architecture
# Source: https://www.kaggle.com/code/devsubhash/visualize-deep-learning-models-using-visualkeras
visualkeras.layered_view(model,
                         legend=True,
                         show_dimension=True,
                         scale_xy=1,                                        # Adjust the scale of the image
                        #  scale_z=1,
                         # to_file='./BaselineModel_Architecture.png',
)

In [ ]:
# Compile model
optimizer = Adam(learning_rate=0.001, weight_decay=0.001, name="Optimizer") #Decreased weight decay for better performance with transfer learning
loss = CategoricalCrossentropy(name="Loss")       
metrics = [CategoricalAccuracy(name="accuracy"), Precision(name="precision"), Recall(name="recall"),
           F1Score(average="macro", name="f1_score"), AUC(name="auc")]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# Create a directory for saving the model and logs
model_name = f"EfficientNetV2S_{datetime.datetime.now().strftime('%Y%m%d')}"                                                                             # Model name
print(f"\n\033[1mModel name:\033[0m {model_name}")

In [ ]:
# Callbacks
model_name = f"EfficientNetV2S_{datetime.datetime.now().strftime('%Y%m%d')}_EfficientNetV2S"                                       # Model name
callbacks = [
    ModelCheckpoint(f"./ModelCallbacks/checkpoint_{model_name}.keras", monitor="val_loss", save_best_only=True, verbose=0), 
    CSVLogger(f"./ModelCallbacks/metrics_{model_name}.csv"),
    LearningRateScheduler(lambda epoch, lr: lr * 0.95), #Change learning rate for better convergence
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True) 
]

In [ ]:
# Train model
start_time = time.time()
history = model.fit(train_datagen, batch_size = batch_size, epochs=10, validation_data=val_datagen, callbacks=callbacks, verbose=1)
train_time = round(time.time() - start_time, 2)

print(f"\nTraining completed in \033[1m{train_time} seconds ({str(datetime.timedelta(seconds=train_time))} h)\033[0m).")

---

### <a class='anchor' id='3_1'></a> <a class='anchor' id='3_2'></a>  **🧪 Model Selection & 📏 Model Evaluation**

In [ ]:
# Evaluate model
from utilities import plot_metrics

os.makedirs("./ModelsEvaluation", exist_ok=True)                                              # Create directory if it doesn't exist
plot_metrics(history, file_path=f"./ModelsEvaluation/2_Training_Validation_Metrics_{datetime.datetime.now().strftime('%Y%m%d')}.png")

In [ ]:
# Evaluate on validation and test sets
train_results = {'accuracy': history.history['accuracy'][-1], 'precision': history.history['precision'][-1], 'recall': history.history['recall'][-1], 'f1_score': history.history['f1_score'][-1], 'auc': history.history['auc'][-1]}
val_results = model.evaluate(val_datagen, batch_size=batch_size, return_dict=True, verbose=1)
test_results = model.evaluate(test_datagen, batch_size=batch_size, return_dict=True, verbose=1)

In [ ]:
# Display results
from utilities import display_side_by_side, create_evaluation_dataframe
results_df = create_evaluation_dataframe(
    model_name="Baseline Model",
    variation="Original | Grayscale=F | Contrast=F | Saturation=F | Adam=0.001",           # Dataset | Grayscale | Contrast | Saturation | Optimizer=Learning Rate
    train_metrics=train_results, val_metrics=val_results, test_metrics=test_results, train_time=train_time,
    csv_save_path= f"./ModelsEvaluation/2_BaselineModel_TrainingValidationMetrics_{model_name}.csv"      # Save the results to a CSV file
)
display_side_by_side(results_df, super_title="Model Evaluation Results")

---

---

# **🔗 Bibliography/References**

**[[1]](https://)** AAAAAAAAAA

---